In [320]:
from sympy import factorint
import math

In [335]:
# Assignment 3 - Pick Numbered Tokens
# Team 33 - Kevin Brousseau - 40060549

# Main function, calculates 1 move of PNT
def nextMove(init, file):
    # Reset Stats class static values
    Stats.nbNodesVisited = 0
    Stats.nbNodesEvaluated = 0
    Stats.maxDepthReached = 0
    # Extract information from input
    integers = init.split()[1:]
    nbTokens = int(integers[0])
    nbTaken = int(integers[1])
    taken = integers[2:2+nbTaken]
    for i in range(len(taken)):
        taken[i] = int(taken[i])
    depth = int(integers[-1])
    # If depth is 0, depth becomes max depth (n)
    if depth == 0:
        depth = nbTokens
    # Define values referenced in if-else
    v = None
    move = None
    # First move
    if nbTaken == 0:
        root = Node(0,None,0)
        v, move = turn1(root, taken, nbTokens, depth, float("-inf"), float("inf"))
    # Subsequent Moves
    else:
        root = Node(int(taken[-1]),None,0)
        taken = taken[:-1]
        if nbTaken % 2 == 0:
            v, move = maxValue(root, taken, nbTokens, depth, float("-inf"), float("inf"))
        else:
            v, move = minValue(root, taken, nbTokens, depth, float("-inf"), float("inf"))
    # Calculate Average Effective Branching Factor
    nonRoot, nonLeaf = avgAEBF(root, 0, 0)
    nonRoot = nonRoot - 1
    # Write output file
    f = open(file+'.txt', "w")
    f.write('Move: '+str(move))
    f.write('\nValue: '+str(round(float(v), 1)))
    f.write('\nNumber of Nodes Visited: '+str(Stats.nbNodesVisited))
    f.write('\nNumber of Nodes Evaluated: '+str(Stats.nbNodesEvaluated))
    f.write('\nMax Depth Reached: '+str(Stats.maxDepthReached))
    f.write('\nAvg Effective Branching Factor: '+str(round(nonRoot / nonLeaf, 1)))

def turn1(root, taken, nbTokens, depth, alpha, beta):
    root.createTurn1Children(nbTokens)
    v = float("-inf")
    move = None
    for child in root.children:
        v2, a2 = minValue(child, taken, nbTokens, depth, alpha, beta)
        if v2 > v:
            v, move = v2, child.token
            alpha = max(alpha, v)
        if v >= beta :
            return v, move
    return v, move

def maxValue(node, taken, nbTokens, depth, alpha, beta):
    node.createChildren(taken, nbTokens)
    taken2 = taken.copy()
    taken2.append(node.token)
    # Evaluation
    if len(node.children) == 0 or node.depth == depth:
        Stats.nbNodesEvaluated = Stats.nbNodesEvaluated + 1
        # At an end game state where Player B (MIN) wins: -1.0 
        if(len(node.children) == 0):
            return -1, None
        # If token 1 is not taken yet, return a value of 0 (because the current state is a relatively neutral
        # one for both players)
        if 1 not in taken2:
            return 0, None
        # If the last move was 1, count the number of the possible successors (i.e., legal moves). If
        # the count is odd, return 0.5; otherwise, return-0.5.
        if taken2[-1] == 1:
            if len(node.children) % 2 == 1:
                return 0.5, None
            else:
                return -0.5, None
        # If last move is a prime, count the multiples of that prime in all possible successors. If the
        # count is odd, return 0.7; otherwise, return-0.7.
        flag = False
        if node.token > 1:
            for i in range(2, node.token):
                if (node.token % i) == 0:
                    flag = True
                    break
        if flag:
            i = 1
            count = 0
            while i * node.token <= nbTokens:
                if i * node.token not in taken2:
                    count = count + 1
                i = i + 1
            if count % 2 == 1:
                return 0.7, None
            else:
                return -0.7, None
        # If the last move is a composite number (i.e., not prime), find the largest prime that can
        # divide last move, count the multiples of that prime, including the prime number itself if it
        # hasn’t already been taken, in all the possible successors. If the count is odd, return 0.6;
        # otherwise, return-0.6.
        else:
            largestPrime = max(factorint(node.token).keys())
            i = 1
            count = 0
            while i * largestPrime <= nbTokens:
                if i * largestPrime not in taken2:
                    count = count + 1
                i = i + 1
            if count % 2 == 1:
                return 0.6, None
            else:
                return -0.6, None
    v = float("-inf")
    move = None
    for child in node.children:
        v2, a2 = minValue(child, taken2, nbTokens, depth, alpha, beta)
        if v2 > v:
            v, move = v2, child.token
            alpha = max(alpha, v)
        #print('token:',move,'score:',v,'depth',child.depth)
        if v >= beta :
            return v, move
    return v, move

def minValue(node, taken, nbTokens, depth, alpha, beta):
    node.createChildren(taken, nbTokens)
    taken2 = taken.copy()
    taken2.append(node.token)
    # Evaluation
    if len(node.children) == 0 or node.depth == depth:
        Stats.nbNodesEvaluated = Stats.nbNodesEvaluated + 1
        # At an end game state where Player A (MAX) wins: 1.0 
        if(len(node.children) == 0):
            return 1, None
        # If token 1 is not taken yet, return a value of 0 (because the current state is a relatively neutral
        # one for both players)
        if 1 not in taken2:
            return 0, None
        # If the last move was 1, count the number of the possible successors (i.e., legal moves). If
        # the count is odd, return -0.5; otherwise, return 0.5.
        if taken2[-1] == 1:
            if len(node.children) % 2 == 1:
                return -0.5, None
            else:
                return 0.5, None
        # If last move is a prime, count the multiples of that prime in all possible successors. If the
        # count is odd, return -0.7; otherwise, return 0.7.
        flag = False
        if node.token > 1:
            for i in range(2, node.token):
                if (node.token % i) == 0:
                    flag = True
                    break
        if flag:
            i = 1
            count = 0
            while i * node.token <= nbTokens:
                if i * node.token not in taken2:
                    count = count + 1
                i = i + 1
            if count % 2 == 1:
                return -0.7, None
            else:
                return 0.7, None
        # If the last move is a composite number (i.e., not prime), find the largest prime that can
        # divide last move, count the multiples of that prime, including the prime number itself if it
        # hasn’t already been taken, in all the possible successors. If the count is odd, return -0.6;
        # otherwise, return 0.6.
        else:
            largestPrime = max(factorint(node.token).keys())
            i = 1
            count = 0
            while i * largestPrime <= nbTokens:
                if i * largestPrime not in taken2:
                    count = count + 1
                i = i + 1
            if count % 2 == 1:
                return -0.6, None
            else:
                return 0.6, None
    v = float("inf")
    move = None
    for child in node.children:
        v2, a2 = maxValue(child, taken2, nbTokens, depth, alpha, beta)
        if v2 < v:
            v, move = v2, child.token
            beta = min(beta, v)
        #print('token:',move,'score:',v,'depth',child.depth)
        if v <= alpha :
            return v, move
    return v, move

def avgAEBF(node, nonRoot, nonLeaf):
    flag = False
    for child in node.children:
        if child.explored == True:
            flag = True
            nonRoot, nonLeaf = avgAEBF(child, nonRoot, nonLeaf) 
    if flag == True:
        nonRoot = nonRoot + 1
        nonLeaf = nonLeaf + 1
    
    if flag == False:
        return (nonRoot + 1, nonLeaf);
            
    return (nonRoot, nonLeaf)

class Stats:
    nbNodesVisited = 0
    nbNodesEvaluated = 0
    maxDepthReached = 0

class Node:
    def __init__(self, token, parent, depth):
        self.token = token
        self.parent = parent
        self.depth = depth
        self.children = []
        self.explored = False
    def setChildren(children):
        self.children = children
    def __str__(self):
        return str(self.token)
    def createChildren(self, taken, nbTokens):
        Stats.nbNodesVisited = Stats.nbNodesVisited + 1
        Stats.maxDepthReached = max(Stats.maxDepthReached, self.depth)
        self.explored = True
        children = []
        taken2 = taken.copy()
        currentParent = self
        while currentParent != None:
            taken2.append(currentParent.token)
            currentParent = currentParent.parent
            
        for i in range(1,nbTokens+1):
            if i % self.token == 0 and i not in taken2 or self.token % i == 0 and i not in taken2:
                child = Node(i, self, self.depth+1)
                self.children.append(child)
    def createTurn1Children(self, nbTokens):
        Stats.nbNodesVisited = Stats.nbNodesVisited + 1
        self.explored = True
        for i in range(nbTokens):
            if (i+1) % 2 == 1 and i+1 < nbTokens/2:
                child = Node(i+1, self, self.depth+1)
                self.children.append(child)

In [343]:
# myinput = 'TakeTokens 30 7 3 1 8 4 2 10 20 12'
# nextMove(myinput, 'output0')
f = open("randomCases.txt","r")
i = 1
for line in f:
    fileName = 'output' + str(i)
    nextMove(line, fileName)
    i = i+1
